<a href="https://colab.research.google.com/github/seanngpack/machine-learning-sets/blob/master/Heart_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's do some more deep learning

In [0]:
# import the basics
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [0]:
# import the dataset
from google.colab import drive
drive.mount('/content/gdrive')
# 4/egFjfdz2RbDdyEUUKDdaX9I2ts_Cvj25RA39CaDEPpATO-zzV6XgMms

In [0]:
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Datasets/heart.csv')

In [240]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [0]:
import seaborn as sns
sns.pairplot(df, hue='cp')

## Preprocess the data

In [0]:
# import some preprocessing tools
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

In [243]:
y = df['target']
y[:5]

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64

In [244]:
X = df.drop(columns=['target'])
X.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


In [0]:
# We gotta scale the features
from sklearn.preprocessing import StandardScaler

In [0]:
sc = StandardScaler()
Xsc = sc.fit_transform(X) # this transform converts from df -> numpy array

### Making train/test set split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(Xsc, y,
                                                    test_size=0.2,
                                                    random_state=42)

In [248]:
X_train.shape

(242, 13)

## Build the model

In [0]:
# Import the model building tools
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam, Adadelta, RMSprop
import keras.backend as K

In [0]:
def build_model():
  K.clear_session()
  model = Sequential()
  model.add(Dense(16, input_shape=(13, ),                  
                  activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(RMSprop(lr=0.1),
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [251]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                224       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________


In [252]:
model.fit(X_train, y_train,
          batch_size=10,
          epochs=20,
          verbose=1,
          validation_split=0.2)

Train on 193 samples, validate on 49 samples
Epoch 1/20
193/193 [==============================] - 0s 1ms/step - loss: 0.6197 - acc: 0.7150 - val_loss: 0.4855 - val_acc: 0.7143
Epoch 2/20
193/193 [==============================] - 0s 153us/step - loss: 0.5088 - acc: 0.7772 - val_loss: 0.3938 - val_acc: 0.7959
Epoch 3/20
193/193 [==============================] - 0s 150us/step - loss: 0.3534 - acc: 0.8187 - val_loss: 1.5247 - val_acc: 0.5510
Epoch 4/20
193/193 [==============================] - 0s 161us/step - loss: 0.3680 - acc: 0.8446 - val_loss: 0.4912 - val_acc: 0.8571
Epoch 5/20
193/193 [==============================] - 0s 150us/step - loss: 0.3449 - acc: 0.8601 - val_loss: 0.9029 - val_acc: 0.7551
Epoch 6/20
193/193 [==============================] - 0s 184us/step - loss: 0.3707 - acc: 0.8342 - val_loss: 1.0836 - val_acc: 0.7347
Epoch 7/20
193/193 [==============================] - 0s 165us/step - loss: 0.3600 - acc: 0.8549 - val_loss: 0.6420 - val_acc: 0.7959
Epoch 8/20
193/193 

In [253]:
model.evaluate(X_test, y_test)

61/61 [==============================] - 0s 138us/step


[2.1935567738579924, 0.8360655757247425]

In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
# fix this
y_pred = model.predict(X_test)
y_pred_class = (y_pred > 0.5).astype(np.int)
# argmax returns the index
y_test_class = y_test.values


In [256]:
matrix = confusion_matrix(y_test_class, y_pred_class) 
matrix

array([[25,  4],
       [ 6, 26]])

In [261]:
accuracy = (matrix[0,0] + matrix[1,1]) / np.sum(matrix.reshape(1,4), axis=1)[0]
print(str(accuracy*100) + '% accuracy')

83.60655737704919% accuracy
